In [ ]:
from IPython.display import display


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
import csv
import math
from collections import defaultdict

directions_order = ["Marseille", "Bordeaux", "Rouen", "Nantes",  "La Rochelle", "Bayonne"]

data = []
with open("evolution_directions_fermes_local.csv", "r") as data_f, open("regressions_local.csv", "r") as reg_data_f:
    reg_data = defaultdict(dict)
    for f in list(csv.DictReader(reg_data_f)):
        reg_data[f['direction_ferme']][f['kind']] = f
    for d in csv.DictReader(data_f):
        slope = float(reg_data[d['direction_ferme']][d['kind']]['slope'])
        data.append({
            "year": d['year'],
            "value": float(d['value']), 
            "kind" : d['kind'],
            "slope" : f"{d['direction_ferme']} {'+' if slope>0 else ''}{slope*100:0.1f}%/an",
            "reg_point": math.exp(slope*float(d['year'])+float(reg_data[d['direction_ferme']][d['kind']]['intercept'])) ,
            "direction_ferme": d['direction_ferme'],
            "column_order": directions_order.index(d['direction_ferme']) })



In [ ]:
data

In [ ]:

  VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Comparaison du dynamisme des principaux ports français après 1740 (1787 excepté)",
    "data": {"values": data},
    "facet":{
      "column" : {"field": "direction_ferme", 
                  "header": {"titleFontSize": 0, "labelFontSize": 20},
                  "sort": {"field": "column_order", "order": "ascending"}},
      "row": {"field" : "kind"}
    },
    "spec":{
    "layer": [
      {
        "mark": {
          "type": "point",
          "filled": True
        },
        "encoding": {
          "x": {
            "field": "year",
            "type": "temporal",
            "scale": {"domain": ["1740","1789"]}
          },
          "y": {
            "field": "value",
            "type": "quantitative"
          }
        }
      },
      {
        "mark": {
          "type": "line",
          "color": "firebrick"
        },
        "encoding": {
          "x": {
            "field": "year",
            "type": "temporal",
            "scale": {"domain": ["1750","1780"]}
          },
          "y": {
            "field": "reg_point",
            "type": "quantitative"
          }
        }
      },
      {
      
      "mark": {
        "type": "text",
        "color": "firebrick",
        "x": "0",
        "align": "left",
        "y": 15,
        "fontSize": 15,
        "font": "sans serif"
      },
      "encoding": {
        "text": {"type": "nominal", "field": "slope"}
      }
    },

    ]}
  })